In [ ]:
keyword = input("궁금하신 분야의 키워드를 입력해주세요! : ")
print("입력하신 키워드는 ",keyword," 입니다. \n해당 키워드로 면접 질문을 탐색합니다.")

# 원래 데이터 가져와서 전처리

In [ ]:
import pandas as pd
def get_df(file): #파일명을 입력값으로 받는다.
    interview_data=pd.read_excel(file)
    return interview_data

In [ ]:
import pandas as pd
interview_data=get_df('/content/drive/MyDrive/Colab Notebooks/KHUDA_7th_YB_ToyProject/data/merged_interniew_category.xlsx')
interview_data  #증강 + 메인 데이터셋

,Question,Category
0,A/B Test 등 현상 분석 및 실험 설계 상 통계적으로 유의미함의 여부를 결정하...,데이터 분석가
1,Agile 방법론이 무엇인지 설명해주세요,데이터 분석가
2,Bias를 통제하는 방법은 무엇입니까?,데이터 분석가
3,Java에서 Builder 패턴을 사용하는이유는?,데이터 분석가
4,Java에서 팩토리 메서드 패턴을 사용하는 이유는?,데이터 분석가
...,...,...
2901,Event Delegation에 대해 설명해주십시오.,프론트엔드
2902,PWA에 대해 말씀해주세요.,프론트엔드
2903,웹 저장소의 종류와 특징을 설명해 보세요.,프론트엔드
2904,웹 워커에 대해서 말해주세요.,프론트엔드


In [ ]:
!pip install konlpy
from konlpy.tag import Okt #konlpy의 Okt 형태소 분석기를 받아온 후 문장이 들어오면 어근을 분리하는 함수 생성, konlpy패키지 별도 설치 필요


okt=Okt()
def okt_tokenizer(sentence):
    okt_token=okt.morphs(sentence) #.morphs로 텍스트를 형태소 단위로 나눈다
    return okt_token


In [ ]:
question_cleasinged=interview_data['Question'].values.astype('str') #면접 질문 데이터(interview_data)의 question컬럼의 데이터 타입을 str로 변경

In [ ]:
question_cleasinged

array(['A/B Test 등 현상 분석 및 실험 설계 상 통계적으로 유의미함의 여부를 결정하기 위한 방법에는 어떤 것이 있을까요?',
       'Agile 방법론이 무엇인지 설명해주세요', 'Bias를 통제하는 방법은 무엇입니까?', ...,
       '웹 저장소의 종류와 특징을 설명해 보세요.', '웹 워커에 대해서 말해주세요.',
       '웹 컴포넌트에 대해 말씀해주세요.'], dtype='<U192')

In [ ]:
#불용어 처리
stop_words=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KHUDA_7th_YB_ToyProject/data/korean_stopwords.txt') #불용어 처리를 위해 korean_stopwords.txt의 파일을 불러온다.
stop_words_interview=stop_words['불용어'].tolist() #불용어 리스트를 생성

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer #Tfidf벡터화를 통해 각각의 어근을 피쳐로 갖는 희소행렬을 생성, stop_words 파라미터를 통해 불용어 제거

tfidf=TfidfVectorizer(tokenizer=okt_tokenizer,ngram_range=(1,2),stop_words=stop_words_interview ,lowercase=False,) #파라미터는 추후 모델 성능을 비교하며 조정
tfidf.fit(question_cleasinged)
tfidf_interview_data=tfidf.transform(question_cleasinged)
tfidf_interview_data

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['\t', '가서', '각하', '간', '같다면', '같은', '같이', '거나', '걸', '견지', '결론', '고려', '곳', '공동', '관', '관계', '관련', '구체', '그러한', '그런', '그럼에도', '그렇게', '그치지', '근거', '기대', '기점', '기준', '김', '까닭', '나머지', '남', '낫다', '낼', '년도', '누가', '다', '다시', '달려', '답', '대로', '대하', '더', '더욱이', '도', '동시', '됏', '되는', '되다', '되면', '되어', '된', '된다', '될것이다', '두번째', '뒤', '든', '든지', '들면', '들자면', '듯', '따르는', '따름', '때문', '라', '라도', '라면', '리하', '마자', '막론', '만', '만으로', '만은', '만이', '만큼', '만하', '많은', '말', '면', '모른다', '몰라도', '몰랏다', '못', '바', '바꿔', '밖에', '반대', '방면', '번째', '보다', '보아', '본대', '불구', '비', '비교', '비길수', '비추어', '뿐', '상대', '서', '서술', '수', '시작', '시초', '쓰여

<2906x14483 sparse matrix of type '<class 'numpy.float64'>'
	with 46189 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_interview_data.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#키워드 전처리

In [ ]:
keyword_cleasinged = [str(keyword)]
print(keyword_cleasinged)
print(okt_tokenizer(keyword))

['LSTM과 곡선  함수 커브 SVM 텍스트 딥러닝']
['LSTM', '과', '곡선', '함수', '커브', 'SVM', '텍스트', '딥', '러닝']


In [ ]:
tfidf_keyword_data=tfidf.transform(keyword_cleasinged)
print("tfidf_keyword_data.shape : ", tfidf_keyword_data.shape)
print("tfidf_keyword_data.toarray() : ", tfidf_keyword_data.toarray())

tfidf_keyword_data.shape :  (1, 14483)
tfidf_keyword_data.toarray() :  [[0. 0. 0. ... 0. 0. 0.]]


#키워드 모델 학습

In [ ]:
#svm 모델 불러오기
import pickle

with open("/content/drive/MyDrive/Colab Notebooks/KHUDA_7th_YB_ToyProject/modeling/best_svm_model.pkl", "rb") as f:
    loaded_svm = pickle.load(f)

In [ ]:
y_pred = loaded_svm.predict(tfidf_keyword_data.toarray())
print("예측 결과:", y_pred)

예측 결과: [1]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [ ]:
#해당 분야의 질문만 추출하기
# 타겟 값이 3인 데이터의 인덱스 찾기
import numpy as np
target_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/KHUDA_7th_YB_ToyProject/data/merge_target.csv")
target_data = target_data[:1453]
print(target_data.shape)
print(target_data.head())
indices = np.where(target_data == y_pred[0])[0]

print("타겟 값이",y_pred[0], "인 데이터의 개수:", len(indices))
print("타겟 값이",y_pred[0], "인 데이터의 인덱스:", indices[0]," ~ ",indices[len(indices)-1])
first_index = indices[0]
last_index = indices[len(indices)-1]

(1453, 1)
   0
0  0
1  0
2  0
3  0
4  0
타겟 값이 1 인 데이터의 개수: 245
타겟 값이 1 인 데이터의 인덱스: 140  ~  384


In [ ]:
from sklearn.preprocessing import LabelEncoder
interview_data=get_df('/content/drive/MyDrive/Colab Notebooks/KHUDA_7th_YB_ToyProject/data/merged_interniew_category.xlsx')
category=interview_data['Category'].values
# LabelEncoder를 객체로 생성한 후 , fit( ) 과 transform( ) 으로 label 인코딩 수행.
encoder = LabelEncoder()
encoder.fit(category)#학습
category_encoded = encoder.transform(category)#숫자로 변환
print('인코딩 변환값:',category_encoded)
interview_data['Category'].value_counts().index
print('인코딩 클래스:',encoder.classes_)
print('디코딩 원본 값:',encoder.inverse_transform([0,1,2,3,4]))
def category_encoding(df, category_index): #입력값으로 데이터 프레임과 인코딩할 컬럼의 인덱스를 받는다.
    category=df.iloc[:,category_index].values
    encoder = LabelEncoder()
    encoder.fit(category)
    category_encoded = encoder.transform(category)
    return category_encoded
interview_data['category_encoded']=category_encoded

인코딩 변환값: [0 0 0 ... 4 4 4]
인코딩 클래스: ['데이터 분석가' '데이터 사이언티스트' '데이터 엔지니어' '백엔드' '프론트엔드']
디코딩 원본 값: ['데이터 분석가' '데이터 사이언티스트' '데이터 엔지니어' '백엔드' '프론트엔드']


In [ ]:
#해당 카테고리 질문들 모음
category_data = tfidf_interview_data[first_index:last_index + 1]
category_target = pd.DataFrame(category_encoded)[first_index:last_index + 1]
print(category_data.shape)
print(category_target.shape)

(245, 14483)
(245, 1)


In [ ]:
main_question=interview_data['Question']
category_question = interview_data['Question'][first_index : last_index]

#유사도측정

In [ ]:
#면접 질문과 유사도 측정
#유사도 측정은 (면접질문 + 분야)로 진행,index 추출
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 키워드와 나머지 문장들의 코사인 유사도 계산
cosine_sim_text1 = cosine_similarity(tfidf_keyword_data,category_data).flatten()

# 가장 유사한 문장의 인덱스 찾기
most_similar_text_index = np.argmax(cosine_sim_text1)  # 가장 높은 유사도를 가진 인덱스

# 주어진 텍스트에서 가장 유사한 문장 출력
most_similar_text_to_text1 = main_question[most_similar_text_index]

print("키워드 기준으로 가장 유사한 문장:", most_similar_text_to_text1)

# 해당 문장과 키워드 간의 코사인 유사도 출력
cosine_sim_between_texts = cosine_sim_text1[most_similar_text_index]
print("키워드와 가장 유사한 문장 간의 코사인 유사도:", cosine_sim_between_texts)

키워드 기준으로 가장 유사한 문장: D를 학습할때 G의 Weight을 고정해야 한다. 방법은?
키워드와 가장 유사한 문장 간의 코사인 유사도: 0.3936124946535703


In [ ]:
#유사도 높은 순으로 나열
#면접질문만으로 진행


# 유사도가 높은 순으로 정렬된 인덱스 가져오기
sorted_indices = np.argsort(cosine_sim_text1)[::-1]  # 내림차순 정렬

# 상위 N개 결과 출력 (예: 상위 5개)
top_n = 10
print(f"keyword와 가장 유사한 {top_n}개의 문장")

for i in range(top_n):
    idx = sorted_indices[i]  # 정렬된 인덱스에서 i번째 가져오기
    print(f"{i+1}. {main_question[idx]} (유사도: {cosine_sim_text1[idx]:.4f})")

keyword와 가장 유사한 5개의 문장
1. D를 학습할때 G의 Weight을 고정해야 한다. 방법은? (유사도: 0.3936)
2. 형상관리란? (유사도: 0.3300)
3. 데이터 분석 관련 창업 경험을 설명해주세요. (유사도: 0.2328)
4. ROC 커브에 대해 설명해주실 수 있으신가요? (유사도: 0.2207)
5. 베이지안과 프리퀀티스트간의 입장차이를 설명해주실 수 있나요? (유사도: 0.2090)
